In [1]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import glob
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import random
import cv2
import torch.nn as nn
import torch.nn.functional as F
import sys
device = torch.device("cuda")

In [2]:
class Dataset(object):
    def __getitem__(self, index):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def __add__(self, other):
        return ConcatDataset([self, other])

In [3]:
import glob
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset

class MRI(Dataset):
    def __init__(self):
        self.mode = 'train'
        tumor = []
        healthy = []
        self.x_train, self.x_test, self.y_train, self.y_test = None, None, None, None
        for i in glob.iglob("/archive/brain_tumor_dataset/yes/*.jpg"):
            img = cv2.imread(i)
            img = cv2.resize(img, (128, 128))
            b, g, r = cv2.split(img)
            img = cv2.merge([r, g, b])
            img = img.reshape((img.shape[2], img.shape[0], img.shape[1]))
            tumor.append(img)

        for i in glob.iglob("/archive/brain_tumor_dataset/no/*.jpg"):
            img = cv2.imread(i)
            img = cv2.resize(img, (128, 128))
            b, g, r = cv2.split(img)
            img = cv2.merge([r, g, b])
            img = img.reshape((img.shape[2], img.shape[0], img.shape[1]))
            healthy.append(img)

        tumor = np.array(tumor, dtype=np.float32)
        healthy = np.array(healthy, dtype=np.float32)

        tumor_labels = np.ones(tumor.shape[0], dtype=np.float32)
        healthy_labels = np.ones(healthy.shape[0], dtype=np.float32)

        self.images = np.concatenate((tumor, healthy), axis=0)
        self.labels = np.concatenate((tumor_labels, healthy_labels), axis=0)

    def train_val_split(self):
      if self.images.shape[0] > 0:
          self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(
              self.images, self.labels, test_size=0.2, random_state=42
          )
      else:
          raise ValueError("Dataset is empty. Ensure images are loaded correctly.")



    def __len__(self):
        if self.mode == "train":
            return self.x_train.shape[0]
        if self.mode == "test":
            return self.x_test.shape[0]

    def __getitem__(self, index):
        if self.mode == "train":
            return {"image": self.x_train[index], "label": self.y_train[index]}
        if self.mode == "test":
            return {"image": self.x_test[index], "label": self.y_test[index]}

    def normalize(self):
        self.images = self.images / 255.0




In [4]:

class CNN(nn.Module):
    def __init__(self):
        super(CNN,self).__init__()
        self.cnn_model = nn.Sequential(
        nn.Conv2d(in_channels=3, out_channels=6, kernel_size=5),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2, stride=5),
        nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5),
        nn.Tanh(),
        nn.AvgPool2d(kernel_size=2, stride=5))

        self.fc_model = nn.Sequential(
        nn.Linear(in_features=256, out_features=120),
        nn.Tanh(),
        nn.Linear(in_features=120, out_features=84),
        nn.Tanh(),
        nn.Linear(in_features=84, out_features=1))

    def forward(self, x):
        x = self.cnn_model(x)
        x = x.view(x.size(0), -1)
        x = self.fc_model(x)
        x = F.sigmoid(x)

        return x
    def predicted_value(self, path):

        img = cv2.imread(path)
        img = cv2.resize(img, (128, 128))
        b, g, r = cv2.split(img)
        img = cv2.merge([r, g, b])
        img = img.reshape((img.shape[2], img.shape[0], img.shape[1]))
        img = np.array([img], dtype=np.float32)
        img = img/255.0
        img = torch.from_numpy(img).to(device,dtype=torch.float32)
        # img = img.permute(0, 3, 1, 2)
        # img = img.unsqueeze(0)
        with torch.no_grad():
          self.eval()
          prediction = self(img).squeeze().cpu().numpy()
        return prediction

In [5]:
mri = MRI()
mri.normalize()
mri.train_val_split()

ValueError: Dataset is empty. Ensure images are loaded correctly.

In [ ]:
train_value = DataLoader(mri, batch_size=32, shuffle=True)
test_value = DataLoader(mri, batch_size=32, shuffle=False)

In [ ]:
device = torch.device("cuda")
model = CNN().to(device)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001 )

In [ ]:
epochs_train_loss=[]
epochs_test_loss=[]

In [ ]:
for epoch in range(1, 600):
  train_loss = []
  model.train()
  mri.mode = "train"
  for i in train_value:
    optimizer.zero_grad()
    data = i["image"].to(device)
    label = i["label"].to(device)
    y_hat = model(data)
    loss = nn.BCEWithLogitsLoss()(y_hat.squeeze(), label)
    loss.backward()
    optimizer.step()
    train_loss.append(loss.item())
  epochs_train_loss.append(np.mean(train_loss))

  test_loss = []
  model.eval()
  mri.mode-"test"
  with torch.no_grad():
    for i in test_value:
      data = i["image"].to(device)
      label = i["label"].to(device)
      y_hat = model(data)
      loss = nn.BCEWithLogitsLoss()(y_hat.squeeze(), label)
      loss.backward()
      optimizer.step()
      test_loss.append(loss.item())
  epochs_test_loss.append(np.mean(test_loss))

  if (epoch+1 )%10 == 0:
    print(f'Train Epoch: {epoch} Train Loss: {np.mean(train_loss)} Test Loss: {np.mean(test_loss)}')

In [ ]:
plt.figure(figsize=(16, 9))
plt.plot(epochs_train_loss, c='b', label="Train Loss")
plt.plot(epochs_test_loss, c='r', label="Validation Loss")
plt.legend()
plt.grid()
plt.xlabel("Epochs", fontsize=20)
plt.xlabel("Loss", fontsize=20)

In [ ]:
def have_tumor(value):
  return "Having Brain Tumor" if value>0.5 else "No Brain Tumor"

In [ ]:
img_path = "/content/drive/MyDrive/Brain Tumor/archive/brain_tumor_dataset/no/18 no.jpg"
predict_value = model.predicted_value(img_path)
predict_value, have_tumor(predict_value)

In [ ]:
image_files = glob.iglob('/archive/brain_tumor_dataset/no/10 no.jpg')
print(image_files)
image_files = list(image_files)
print(image_files)
print("   ")
if not image_files:
    print("No image files found.")
else:
    # Read the first image from the list (assuming there's only one)
    image_path = image_files[0]
    img = cv2.imread(image_path)

    # Display the image
    cv2.imshow('Image', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()